# TensorFlow Introduction

In conda prompt, write:
    
    activate tensorflow

to activate the conda environment (to deactivate just write conda deactivate

In [2]:
import tensorflow as tf

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
mnist = input_data.read_data_sets("C:/Users/DQ815GM/MNIST_data/", one_hot=True)

Extracting C:/Users/DQ815GM/MNIST_data/train-images-idx3-ubyte.gz
Extracting C:/Users/DQ815GM/MNIST_data/train-labels-idx1-ubyte.gz
Extracting C:/Users/DQ815GM/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting C:/Users/DQ815GM/MNIST_data/t10k-labels-idx1-ubyte.gz


Let's set our hyper parameters.
The first one is the learning rate which defines how fast we want to update our weights, if the learning rate is too big our model might skip the optimal solution, if it's too small we might need too many iterations to converge on the best results so we'll set it to 0.01 because it's a known decent learning rate for this problem

In [6]:
#set hyper parameters
learning_rate = 0.01
training_iteration = 30
batch_size = 100
display_step = 2

Now we want to create our model in tensorflow a model is represented as a data flow graph the graph contains a set of nodes called operations.

![tensorflow_graph.jpg](images/tensorflow_graph.jpg)

These are units of computation they can be as simple as addition or multiplication and can be complicated at some multivariate equation. Each operation takes in as input a tensor and outputs a tensor as well. A tensor is how data is represented in tensor flow they are multi-dimensional arrays of numbers and they flow between operations hence the name tensor flow.

We start by building our model by creating two operations both our placeholder operations a placeholder is just a variable that we will assign data to at a later date it's never initialized and contains no data well define the type and shape of our data as the parameters.

![placeholder_tensorflow.png](images/placeholder_tensorflow.png)

In [7]:
# TF graph input
x = tf.placeholder(tf.float32, [None, 784]) #mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, 10]) #0-9 digits recognition => 10 classes

In [21]:
#Create a model

We define our weights W and biases b for our model. The weights are the probabilities that affect how data flows in the graph and they will be updated continuously during training, so that our results get closer and closer to the right solution. The bias lets a shift our regression line to better fit.

In [8]:
#Set model weights
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

We create a named scope. Scopes help us organize nodes in the graph visualizer, called tensor board, which will view at the end. We will create three scopes: in the first scope we'll implement our model which is a logistic regression, by matrix multiplying the input images X by the weight matrix W and adding the bias B.

In [9]:
with tf.name_scope('Wx_b') as scope:
    #construct a linear model
    model = tf.nn.softmax(tf.matmul(x, W) + b)

Create summary operations to help us later visualize the distribution of our weights and biases

In [10]:
#Add summury ops to collect data
w_h = tf.summary.histogram('weights', W)
b_h = tf.summary.histogram('biases', b)

In the second scope will create our cost function. The cost function helps us minimize our error during training and we'll use the popular cross-entropy function as it. 
Then we'll create a scalar summary to monitor it during training, so we can visualize it later.

In [11]:
with tf.name_scope('cost_function') as scope:
    #Minimize error using cross entropy
    #Cross entropy
    cost_function = -tf.reduce_sum(y*tf.log(model))
    #Create a summary to monitor the cost function
    tf.summary.scalar('cost_function', cost_function)

Our last scope is called Train and it will create our optimization function that makes our model improve during training. 
We use the popular gradient descent algorithm which takes our learning rate as a parameter for pacing and our cost function as a parameter to help minimize the error.

In [12]:
with tf.name_scope('train') as scope:
    #Gradient descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

Now that we have our graph built, we will initialize all of our variables then we'll merge all of our summaries into a single operator.

In [13]:
# Initializing the variables
init = tf.initialize_all_variables()

#Merge all summaries into a single operator
merged_summary_op = tf.summary.merge_all()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Now we're ready to launch our graph by initializing a session which lets us execute our data flow graph. Then we set our summary write or folder location which will later load data to visualize in tensor board.
We got to the training time, let's set our for loop for our specified number of iterations and initialize our average cost which will print out every so often to make sure our model is improving during training. We compute our batch size and start training over each example in our training data. Next we fit our model using the batch data in the gradient descent algorithm for back propagation. we compute the average loss and write logs for each iteration via the summary writer for each display step we'll display error logs to terminal.

In [14]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Change this to a location on your computer
    summary_writer = tf.summary.FileWriter('C:/Users/DQ815GM/MNIST_data/logs', sess.graph)

    # Training cycle
    for iteration in range(training_iteration):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})
            # Compute the average loss
            avg_cost += sess.run(cost_function, feed_dict={x: batch_xs, y: batch_ys})/total_batch
            # Write logs for each iteration
            summary_str = sess.run(merged_summary_op, feed_dict={x: batch_xs, y: batch_ys})
            summary_writer.add_summary(summary_str, iteration*total_batch + i)
        # Display logs per iteration step
        if iteration % display_step == 0:
            print("Iteration:", '%04d' % (iteration + 1), "cost=", "{:.9f}".format(avg_cost))
    
    
    print("Tuning completed!")
    
    # Test the model
    predictions = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(predictions, "float"))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

Iteration: 0001 cost= 30.051488478
Iteration: 0003 cost= 21.183532762
Iteration: 0005 cost= 20.084198595
Iteration: 0007 cost= 19.705146739
Iteration: 0009 cost= 19.365873906
Iteration: 0011 cost= 19.095534765
Iteration: 0013 cost= 18.947475856
Iteration: 0015 cost= 18.772883054
Iteration: 0017 cost= 18.635726760
Iteration: 0019 cost= 18.549086073
Iteration: 0021 cost= 18.513515310
Iteration: 0023 cost= 18.353174478
Iteration: 0025 cost= 18.223478372
Iteration: 0027 cost= 18.222337539
Iteration: 0029 cost= 18.039090610
Tuning completed!
Accuracy: 0.9222


## TensorBoard

We can then visualize our graph in tensor board. In our browser we'll be able to view the output of our cost function over time under the events tab, under histograms we'll be able to see the variance in our biases and weights over time, under graphs we can view the actual graph we created as well as the variables for weights and bias. We can see the flow of tensors in the form of edges connecting our nodes or operations. We can see each of the three scopes we named in our code earlier and by double clicking on each we can see a more detailed view of how tensors are flowing through each.

By running in the conda prompt:
    
    tensorboard --logdir=C:\Users\DQ815GM\MNIST_data\logs

the prompt will give you a URL where you can see the summaries.
Once you have check everything type CTRL+C in the conda prompt to close that URL.
Then, to end:
    
    conda deactivate